In [ ]:
# download the weaviate client
%pip install weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")


In [ ]:
# create a backup locally
from weaviate.classes.backup import BackupLocation

backup = client.backup.create(
    backup_id="my-backup",
    backend="filesystem", # Using local filesystem for storage
    include_collections=["<COLLECTION_NAME>"], # Which data collections to backup
    wait_for_completion=True, # Code will pause until backup completes
    backup_location=BackupLocation.FileSystem(path="/var/lib/weaviate/backups"),  # Optional path specification
)

print(backup)

In [ ]:
from weaviate.classes.backup import BackupLocation

backup = client.backup.restore(
    backup_id="backup",
    backend="filesystem",
    exclude_collections="<COLLECTION_NAME>",  # Optional, specify collections to exclude
    wait_for_completion=True,
    backup_location=BackupLocation.FileSystem(path="/var/lib/weaviate/backups"),  # Required if a non-default location was used at creation
)

print(backup)